## Backblaze
---

In [ ]:
:dep base64
:dep reqwest = { features = ["blocking"] }
:dep serde_json

In [ ]:
use std::collections::HashMap;

In [ ]:
let api_config: HashMap<String, String> = serde_json::from_str(&std::fs::read_to_string("/secrets/backblaze.json").unwrap()).unwrap();
let api_key = base64::encode(format!("{}:{}", api_config["key_id"], api_config["app_key"]));
let auth_url = "https://api.backblazeb2.com/b2api/v2/b2_authorize_account";
let b2 = reqwest::blocking::Client::new();
let auth_req = b2.get(auth_url).header("Authorization", format!("Basic {api_key}")).build().unwrap();
let auth_resp = b2.execute(auth_req).unwrap().text().unwrap();
let auth: HashMap<String, serde_json::Value> = serde_json::from_str(&auth_resp).unwrap();
let auth_token = auth["authorizationToken"].clone().as_str().unwrap().to_string();
let download_url = auth["downloadUrl"].clone().as_str().unwrap().to_string();

In [30]:
let download_req = b2.get(format!("{download_url}/file/digisafe/00asusjr/digisafe.db"))
    .header("Authorization", format!("{auth_token}"))
    .build().unwrap();
let db = b2.execute(download_req).unwrap().text().unwrap();

In [31]:
std::fs::write("/secrets/digisafe_00asusjr.db", db);